# The convolutif model like the deep learning

In [ ]:
import tensorflow as tf
import numpy as py
from tensorflow.examples.tutorials.mnist import input_data

# The ModelConvolutif model

In [ ]:
class ModelConvolutif():

    def __init__(self,tailImg,ncol,nclasses,nhidden):

        tf.reset_default_graph()
        self.tailImg,self.ncol=tailImg,ncol
        self.x=tf.placeholder(tf.float32,shape=[None, tailImg,tailImg,ncol])
        self.y=tf.placeholder(tf.float32,shape=[None,nclasses])
        self.prob=tf.placeholder(tf.float32)
        self.filtr,self.conv,self.b,self.nhidden,self.nclas=None,None,None,nhidden,nclasses
        self.tf_session,self.t=tf.Session(),0

    def convolution(self,taille=5,nfiltr=100):
        if self.t==0 :
            self.filtrage(taille,nfiltr)
            self.conv=tf.nn.conv2d(self.x,self.filtr,strides=[1,1,1,1],padding="SAME") +self.b
        else :
            self.filtrage(taille,nfiltr)
            self.conv=tf.nn.conv2d(self.conv,self.filtr,strides=[1,1,1,1],padding="SAME") +self.b
        self.conv=tf.nn.relu(self.conv)
        self.maxPool()

    def filtrage(self,taille=5,nfiltr=50):
        if self.t==0 :
            p=int(self.x.get_shape()[-1])
        else :
            p=int(self.conv.get_shape()[-1])
        self.t=1
        self.filtr=tf.Variable(tf.truncated_normal(shape=(taille,taille,p,nfiltr)))
        self.b=tf.Variable(tf.zeros([nfiltr]))

    def maxPool(self):
        if self.t==1:
            self.conv=tf.nn.max_pool(self.conv,strides=[1,2,2,1],ksize=[1,2,2,1],padding="SAME")

    def getFlatten(self):
        if self.t==0:
            return None
        return tf.contrib.layers.flatten(self.conv)

    def train(self,lr,sizet,sizebatch,datatrain):
        if self.t==0:
            return None
        vecteur=self.getFlatten()
        self.w1=tf.Variable(tf.truncated_normal(shape=[int(vecteur.get_shape()[-1]),self.nhidden]))
        self.b1=tf.Variable(tf.zeros([self.nhidden]))
        self.h1=tf.nn.relu(tf.matmul(vecteur,self.w1) +self.b1)
        self.w2=tf.Variable(tf.truncated_normal(shape=[self.nhidden,self.nclas]))
        self.b2=tf.Variable(tf.zeros([self.nclas]))
        self.h2=tf.matmul(self.h1,self.w2) +self.b2
        self.soft=tf.nn.softmax(self.h2)
        self.cross_entropy=tf.nn.softmax_cross_entropy_with_logits(labels=self.y,logits=self.h2)
        self.error=tf.reduce_mean(self.cross_entropy)
        eq=tf.equal(tf.argmax(self.soft,1),tf.argmax(self.y,1))
        self.accy=tf.reduce_mean(tf.cast(eq,"float"))
        self.optim=tf.train.AdamOptimizer(lr).minimize(self.error)
        self.tf_session.run(tf.global_variables_initializer())
        self.acc=[]
        self.erre=[]
        for i in range(sizet):
            images,labels=datatrain.next_batch(sizebatch)
            images=images.reshape(sizebatch,self.tailImg,self.tailImg,self.ncol)
            d={self.x:images,self.y:labels,self.prob:0.5}
            _,a,er,r=self.tf_session.run([self.optim,self.accy,self.error,self.x],feed_dict=d)
            self.acc.append(a)
            self.erre.append(er)
            
    def predict(self,image):
        h=self.tf_session.run(self.soft,feed_dict={self.x:[image.reshape(self.tailImg,self.tailImg,self.ncol)]})
        return py.argmax(h)

In [ ]:

data=input_data.read_data_sets("input/data",one_hot=True)


In [ ]:
convol=ModelConvolutif(28,1,10,1)
convol.convolution(8,32)
convol.convolution(5,64)

In [ ]:
convol.train(0.1,100,100,data.train)

In [ ]:
convol.acc

In [ ]:
convol.predict(data.train.images[5])

In [ ]:
data.train.labels[5]